# Projeto | Análise de Texto e Fontes Desestruturadas Web

## Feito por: Beni Kracochansky | Gabriel Kabbani | Rodrigo Mattar

A ideia do projeto é fazer um código que, dado a url do pdf do 10K de uma empresa listada americana, consiga baixar esse pdf e então fazer uma análise gráfica do faturamento e EBITDA da companhia em questão. 

Para testar o funcionamento do projeto, utilizamos o 10K (annual fillings para a SEC) da Apple.


In [11]:
import requests


# PDF para imagem
from pdf2image import convert_from_path

# para trabalhar com diretórios / sistema operacional
import os

# para os DataFrames
import pandas as pd

# para exibir os arquivos de imagem
import cv2
import matplotlib.pyplot as plt

#OCR
import pytesseract

### Baixando Arquivo e Salvando como PDF:

In [12]:

data_url = "https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/b4266e40-1de6-4a34-9dfb-8632b8bd57e0.pdf" #link para o 10k da Apple

nome_saida = 'apple10k.pdf'

def baixa(pdf_url, nome_saida):
    response = requests.get(pdf_url) #pegando o pdf
    
    response.raise_for_status()  # vendo se está válido

    with open(nome_saida, 'wb') as output_file:
    
        output_file.write(response.content) #escrevendo arquvio de saida

baixa(data_url, nome_saida)

### Salvando PDF como imagem

In [13]:
paginas = convert_from_path('apple10k.pdf', 200)


In [14]:
#Salvando cada pagina do PDF como imagem para extracao usando OCR
i = 1
lista_imagens = []
for pg in paginas:
    img_path = './img/Apple10k_{}.jpg'.format(i)
    pg.save(img_path, 'JPEG')
    lista_imagens.append(img_path)
    i += 1

### Extraindo os dados das imagens e salvando no arquivo txt


In [15]:
with open('./text/texto.txt', 'w') as f:
    for image_path in lista_imagens:
        try:
            text = pytesseract.image_to_string(image_path)
            str_texto = "".join(text)
            f.write(str_texto)
        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")
